In [1]:
import pandas as pd
import numpy as np
from os import listdir

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random
import math

In [2]:
def HeaderMaker(PatientFile):
    df_cleaned = pd.read_csv('/data/ortho/' + PatientFile, header=None)

    df_cleaned = df_cleaned.rename(columns={0: "thorax_r_x", 1: "thorax_r_y", 2: "thorax_r_z"})
    df_cleaned = df_cleaned.rename(columns={3: "clavicula_r_x", 4: "clavicula_r_y", 5: "clavicula_r_z"})
    df_cleaned = df_cleaned.rename(columns={6: "scapula_r_x", 7: "scapula_r_y", 8: "scapula_r_z"})
    df_cleaned = df_cleaned.rename(columns={9: "humerus_r_x", 10: "humerus_r_y", 11: "humerus_r_z"})
    df_cleaned = df_cleaned.rename(columns={12: "ellebooghoek_r"})
    df_cleaned = df_cleaned.rename(columns={15: "thorax_l_x", 16: "thorax_l_y", 17: "thorax_l_z"})
    df_cleaned = df_cleaned.rename(columns={18: "clavicula_l_x", 19: "clavicula_l_y", 20: "clavicula_l_z"})
    df_cleaned = df_cleaned.rename(columns={21: "scapula_l_x", 22: "scapula_l_y", 23: "scapula_l_z"})
    df_cleaned = df_cleaned.rename(columns={24: "humerus_l_x", 25: "humerus_l_y", 26: "humerus_l_z"})
    df_cleaned = df_cleaned.rename(columns={27: "ellebooghoek_l"})
    
    return df_cleaned

In [3]:

Patients = {}
testdataurl = '/data/ortho/'
for PatientFile in listdir(testdataurl):
    if ('meting' not in PatientFile):
        print('Skipping: %s' % PatientFile)
        # Skip this iteration
        continue
    Splitted = PatientFile.split('_')
    Patient = Splitted[0] + '_' + Splitted[1]

    Data = HeaderMaker(PatientFile)
    MaxDict = Data.max().to_dict() # .abs() for absolute numbers
    
    if(Patient not in Patients.keys()):
        Patients[Patient] = MaxDict
        Patients[Patient]['GroundTruth'] = Splitted[0].split('Cat')[1]
        Patients[Patient]['Bias'] = 0
        continue
    
    for key in MaxDict.keys():
        Patients[Patient][key]= max(Patients[Patient][key],MaxDict[key])
        

    
    
            
            
            
            

Skipping: StudentNummers
Skipping: testset
Skipping: AllPatients.csv
Skipping: TestDataAllPatients.csv


In [4]:
df = pd.DataFrame.from_dict(Patients, orient='index')
# print(df.columns.values)
df = df[['Cat2' in index or 'Cat3' in index for index in df.index]] # Welke colommen wil je houden

df['TrueFalse'] = ['2' == i for i in df['GroundTruth']]

# print(df.head())

In [5]:
Xcolumns = ['Bias']
Xcolumns.extend(['thorax_r_x', 'thorax_r_y', 'thorax_r_z', 'clavicula_r_x', 'clavicula_r_y',
 'clavicula_r_z', 'scapula_r_x', 'scapula_r_y', 'scapula_r_z', 'humerus_r_x',
 'humerus_r_y', 'humerus_r_z', 'ellebooghoek_r', 'thorax_l_x',
 'thorax_l_y', 'thorax_l_z', 'clavicula_l_x', 'clavicula_l_y', 'clavicula_l_z',
 'scapula_l_x', 'scapula_l_y', 'scapula_l_z', 'humerus_l_x', 'humerus_l_y',
 'humerus_l_z', 'ellebooghoek_l'])

# X = df[Xcolumns]
# y = df[['GroundTruth']]




In [6]:
print(df['GroundTruth'].unique())

['2' '3']


In [7]:
def Patient_test_split(dataframe, xcol, ycol, Percentage):
    allIndex = np.unique(df.index.tolist()) # Get all unique patients id's
    random.seed(2) # Set random seeed so the answer is the same
    
    PercentageIndex = []
    for i in df['GroundTruth'].unique(): # Cycle through each categorie (only cycles through the ones that are present)
        CatPatients = np.array([i.split('_')[0].split('Cat')[1] for i in allIndex])
        CatPatients = CatPatients[CatPatients==i]
        
        AmountItems = len(CatPatients)
        AmountRandom = math.floor(AmountItems*Percentage)
        PercentageIndex.extend(random.sample(list(CatPatients), AmountRandom))        
        
    AmountItems = len(allIndex)
    AmountRandom = math.floor(AmountItems*Percentage)
    
    PercentageIndex = random.sample(list(allIndex), AmountRandom)
    
    Percentagedf = df[xcol][df.index.isin(PercentageIndex)]
    Percentagey = df[ycol][df.index.isin(PercentageIndex)]
    
    Testdf = df[xcol][~df.index.isin(PercentageIndex)]
    Testy = df[ycol][~df.index.isin(PercentageIndex)]
    print('Final Percentage test: %s' % (len(Testdf)/len(Percentagedf)))
    return (Percentagedf, Testdf, Percentagey, Testy)

X_train, X_test, y_train, y_test = Patient_test_split(df, Xcolumns, 'TrueFalse', 0.85)

Final Percentage test: 0.1896551724137931


In [8]:
print(len(X_train), len(X_test))

58 11


In [9]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [10]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [11]:
y_pred = lr.predict(X_test)
print(lr.coef_[0])
print(metrics.accuracy_score(y_test, y_pred))
print(y_test)

[ 0.         -0.28247975 -0.18311568  0.00763223 -0.04908184 -0.16925349
  0.08036329 -0.35319711  0.12709598 -0.25842076  0.03379824 -0.24479184
  0.02819606  0.00127444 -0.28247975 -0.26941001 -0.11768872  0.01619807
 -0.14433991  0.56571918 -0.06661504  0.33597089 -0.29589912 -0.17565484
  0.18221895  0.12907345  0.0009569 ]
0.8181818181818182
Cat2_pat14     True
Cat2_pat15     True
Cat2_pat18     True
Cat2_pat23     True
Cat2_pat24     True
Cat2_pat35     True
Cat2_pat37     True
Cat3_pat14    False
Cat3_pat18    False
Cat3_pat24    False
Cat3_pat38    False
Name: TrueFalse, dtype: bool


In [12]:
from sklearn.metrics import mean_squared_error, \
recall_score, precision_score, accuracy_score, confusion_matrix

y_true = y_test
y_pred =lr.predict(X_test)

TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()
# print(y_true)
# tabel printen

# print('Informationi matrix %s' % cat)
tab = [["pred pos", TP, FP], ["pred neg", FN, TN]]
print(pd.DataFrame(tab, columns=["", "pos", "neg"]))
print()
print("recall: ", recall_score(y_true, y_pred))
print("precision: ", precision_score(y_true, y_pred))
print("accuracy: ", accuracy_score(y_true, y_pred))
print('\n\n')

             pos  neg
0  pred pos    6    1
1  pred neg    1    3

recall:  0.8571428571428571
precision:  0.8571428571428571
accuracy:  0.8181818181818182





In [13]:
len(Patients['Cat3_pat38'])

32